Base 환경에서 실행

In [4]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [5]:
df_newsgroup = pd.read_csv('C:/Users/user/Desktop/bilm-tf-master/20news_dataset_clear/20newsgroup_preprocessed.csv', sep=';', usecols=['target', 'text_cleaned'])
df_newsgroup.rename(columns={'text_cleaned' : 'text'}, inplace=True)

In [6]:
le = LabelEncoder()
le.fit(df_newsgroup['target'].unique())

LabelEncoder()

In [7]:
df_newsgroup['target'] = le.transform(df_newsgroup['target'])

In [8]:
X = df_newsgroup['text'].astype(str)
y=list(df_newsgroup['target'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=df_newsgroup['target'])

In [9]:
x_train=list(X_train)
x_test=list(X_test)

특수문자 제거

In [10]:
import re
pattern = '[^a-z.0-9 ]'
clr_x_trian=[]
clr_x_test=[]
for sen in x_train:
    clr_x_trian.append(re.sub(pattern, '', sen))
for sen in x_test:
    clr_x_test.append(re.sub(pattern, '', sen))

In [11]:
x_train=clr_x_trian
x_test=clr_x_test

In [12]:
to_txt_filter=x_train+x_test

In [13]:
y=list(y_train)+list(y_test)

문서 길이 조정

In [14]:
#문서 리스트를 입력으로 문서가 설정한 길이(max_words)보다 단어 수가 많다면 뒤의 단어는 truncate하고 새로운 문서 리스트를 출력
def limit_words(text_list, max_words):
    new_list = []
    for text in text_list:
        words = text.split()
        if len(words) > max_words:
            words = words[:max_words]
        new_text = ' '.join(words)
        new_list.append(new_text)
    return new_list

In [15]:
to_txt_filter=limit_words(to_txt_filter,300) #버트 임베딩은 최대 임베딩 토큰 수가 정해져 있으므로 최대길이를 300으로 제한

In [16]:
to_txt_filter

['seen various references triple des recently could anyone tell context would guess means encrypting block times different key time id like sure replies email preferred news unreliable thanks richard pgp public key available request pgp public key available request disclaimer please note personal view construed official comment jet project',
 'j r laferriere wrote wondering law officers read several questions would like ask pertaining motorcycles cops please dont say get vehicle code go local station obvious things like questions would found places answered face face real live flesh cop brother friend cousin whos father cop etc dont bother writing thanks um would mind telling us statemunicipality live different laws different places know tom coradeschi usenet like herd performing elephants diarrhea massive difficult redirect aweinspiring entertaining source mind boggling amounts excrement least expect gene spafford',
 'sandy santra writes mike mattone wrote ive computer months thats lo

버트 모델 정의

In [17]:
import torch
import numpy as np
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM
from itertools import groupby
from operator import itemgetter


tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# 미리 학습된 모델(가중치) 불러오기
model = BertModel.from_pretrained('bert-base-uncased')

In [18]:
len(to_txt_filter)

18828

In [19]:
text = to_txt_filter[5]


marked_text = "[CLS] " + text + " [SEP]"

tokenized_text = tokenizer.tokenize(marked_text) #서브토큰화

indexed_text = tokenizer.convert_tokens_to_ids(tokenized_text)

segments_ids = [1] * len(tokenized_text)

# Python list를 PyTorch tensor로 변환하기 
tokens_tensor = torch.tensor([indexed_text])
segments_tensors = torch.tensor([segments_ids])

with torch.no_grad():
    encoded_layers, _ = model(tokens_tensor, segments_tensors )
    
token_embeddings = torch.stack(encoded_layers, dim=0) #12개 레이어 쌓기 # torch.Size([12, 1, 22, 768])
token_embeddings = torch.squeeze(token_embeddings, dim=1) #배치 차원 제거 # torch.Size([12, 22, 768])
token_embeddings = token_embeddings.permute(1,0,2) #위치 변환 # torch.Size([22, 12, 768])

token_vecs_sum = []

for token in token_embeddings:
    
    sum_vec = torch.sum(token[-4:], dim=0)
    
    token_vecs_sum.append(np.array(sum_vec))
    

token_vecs_sum=np.array(token_vecs_sum)

In [20]:
def split_continuous_integers(lst):                                      #연속된 정수리스트를 split 해주는 함수
    for k, g in groupby(enumerate(lst), lambda i_x:i_x[0]-i_x[1]):
        yield list(map(itemgetter(1), g))
        
def add_previous_number(lst):                                            #최초 서브토큰 인덱스 추가
    return [[sub_lst[0] - 1] + sub_lst for sub_lst in lst]

def bert_word_embedding(text):
    marked_text = "[CLS] " + text + " [SEP]"
    tokenized_text = tokenizer.tokenize(marked_text) #서브토큰화
    indexed_text = tokenizer.convert_tokens_to_ids(tokenized_text)
    segments_ids = [1] * len(tokenized_text)
    # Python list를 PyTorch tensor로 변환하기 
    tokens_tensor = torch.tensor([indexed_text])
    segments_tensors = torch.tensor([segments_ids])
    with torch.no_grad():
        encoded_layers, _ = model(tokens_tensor, segments_tensors )
    token_embeddings = torch.stack(encoded_layers, dim=0) #12개 레이어 쌓기 # torch.Size([12, 1, 22, 768])
    token_embeddings = torch.squeeze(token_embeddings, dim=1) #배치 차원 제거 # torch.Size([12, 22, 768])
    token_embeddings = token_embeddings.permute(1,0,2) #위치 변환 # torch.Size([22, 12, 768])
    
    token_vecs_sum = []
    for token in token_embeddings:                #인코더의 마지막 4개의 은닉 상태를 합쳐 최종 벡터(4개 합친 것이 성능이 가장 좋음)
        sum_vec = torch.sum(token[-4:], dim=0)
        token_vecs_sum.append(np.array(sum_vec))
    
    token_vecs_sum=np.array(token_vecs_sum)
    #서브토큰을 결합해 단어에 대한 임베딩 벡터를 얻자
    subword_indices = [i for i, token in enumerate(tokenized_text) if '##' in token]
    index_list = add_previous_number(list(split_continuous_integers(subword_indices))) #index_list는 서브토큰에 해당하는 인덱스를 한 리스트에 묶어줌
    
    new_token_vecs_sum = []
    last_index = 0

    for subword_inx_list in index_list:
        # 이전 인덱스와 현재 인덱스 그룹 사이의 벡터를 추가합니다.
        new_token_vecs_sum.extend(token_vecs_sum[last_index:subword_inx_list[0]])

        # 현재 인덱스 그룹에 해당하는 벡터의 평균을 계산하고 추가합니다.
        avg_vecs = np.mean(token_vecs_sum[subword_inx_list], axis=0)
        new_token_vecs_sum.append(avg_vecs)

        # 마지막 인덱스를 업데이트합니다.
        last_index = subword_inx_list[-1] + 1

    # 마지막 인덱스 이후의 벡터를 추가합니다.
    new_token_vecs_sum.extend(token_vecs_sum[last_index:])

    # 결과를 numpy array로 변환합니다.
    new_token_vecs_sum = np.array(new_token_vecs_sum[1:-1])
    
    return new_token_vecs_sum

In [21]:
token_vecs_sum=bert_word_embedding(text)
token_vecs_sum.shape

(268, 768)

임베딩

In [18]:
except_lst=[]
for i in range(len(x_train)):
    try:
        text = to_txt_filter[i]
        embed_matrix=bert_word_embedding(text)       
        np.save('./bert_embedding/train(bert)/doc%d'%i,embed_matrix)
        print(embed_matrix.shape)
    except:
        except_lst.append(i)   #sub_token 길이가 512가 넘는 문서는 except_lst에 추가

(49, 768)
(84, 768)
(79, 768)
(150, 768)
(49, 768)
(268, 768)
(141, 768)
(160, 768)
(58, 768)
(59, 768)
(149, 768)
(54, 768)
(94, 768)
(58, 768)
(11, 768)
(25, 768)
(65, 768)
(71, 768)
(125, 768)
(300, 768)
(116, 768)
(120, 768)
(129, 768)
(300, 768)
(62, 768)
(71, 768)
(114, 768)
(300, 768)
(29, 768)
(51, 768)
(143, 768)
(66, 768)
(19, 768)
(300, 768)
(67, 768)
(73, 768)
(27, 768)
(114, 768)
(175, 768)
(54, 768)
(300, 768)
(37, 768)
(59, 768)
(59, 768)
(207, 768)
(21, 768)
(118, 768)
(86, 768)
(47, 768)
(14, 768)
(300, 768)
(57, 768)
(50, 768)
(57, 768)
(41, 768)
(69, 768)
(72, 768)
(14, 768)
(63, 768)
(30, 768)
(228, 768)
(15, 768)
(104, 768)
(151, 768)
(58, 768)
(51, 768)
(23, 768)
(103, 768)
(300, 768)
(65, 768)
(120, 768)
(75, 768)
(94, 768)
(95, 768)
(42, 768)
(110, 768)


Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (513 > 512). Running this sequence through BERT will result in indexing errors


(113, 768)
(41, 768)
(15, 768)
(133, 768)
(47, 768)
(36, 768)
(300, 768)
(51, 768)
(110, 768)
(61, 768)


Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (588 > 512). Running this sequence through BERT will result in indexing errors


(105, 768)
(54, 768)
(141, 768)
(92, 768)
(156, 768)
(84, 768)
(88, 768)
(122, 768)
(112, 768)
(133, 768)
(34, 768)
(130, 768)
(144, 768)
(35, 768)
(290, 768)
(26, 768)
(79, 768)
(29, 768)
(300, 768)
(107, 768)
(30, 768)
(47, 768)
(18, 768)
(117, 768)
(16, 768)
(67, 768)
(78, 768)
(44, 768)
(126, 768)
(2, 768)
(64, 768)
(43, 768)
(41, 768)
(39, 768)
(22, 768)
(46, 768)
(136, 768)
(79, 768)
(66, 768)
(256, 768)
(48, 768)
(41, 768)
(81, 768)
(121, 768)
(96, 768)
(19, 768)
(17, 768)
(21, 768)
(51, 768)
(64, 768)
(29, 768)
(22, 768)
(23, 768)
(147, 768)
(195, 768)
(88, 768)
(174, 768)
(44, 768)
(129, 768)
(136, 768)
(46, 768)
(59, 768)
(294, 768)
(10, 768)
(300, 768)
(300, 768)
(300, 768)
(45, 768)
(50, 768)
(33, 768)
(151, 768)
(72, 768)
(16, 768)
(41, 768)
(83, 768)
(67, 768)
(80, 768)
(162, 768)
(10, 768)
(66, 768)
(111, 768)
(48, 768)
(218, 768)
(145, 768)
(124, 768)
(152, 768)
(57, 768)
(46, 768)
(66, 768)
(81, 768)
(300, 768)
(72, 768)
(73, 768)
(97, 768)
(80, 768)
(78, 768)
(123, 76

...

(49, 768)
(223, 768)
(277, 768)
(300, 768)
(75, 768)
(62, 768)
(119, 768)
(68, 768)
(73, 768)
(77, 768)
(29, 768)
(204, 768)
(77, 768)
(13, 768)
(60, 768)
(145, 768)
(42, 768)
(127, 768)
(1, 768)
(121, 768)
(16, 768)
(34, 768)
(56, 768)
(300, 768)
(102, 768)
(86, 768)
(43, 768)
(229, 768)
(38, 768)
(141, 768)
(104, 768)
(108, 768)
(27, 768)
(38, 768)
(18, 768)
(47, 768)
(300, 768)
(103, 768)
(300, 768)
(87, 768)
(77, 768)
(68, 768)
(62, 768)
(85, 768)
(45, 768)
(100, 768)
(118, 768)
(14, 768)
(86, 768)
(235, 768)
(139, 768)
(30, 768)
(29, 768)
(44, 768)
(289, 768)
(148, 768)
(154, 768)
(43, 768)
(57, 768)
(300, 768)
(251, 768)
(83, 768)
(283, 768)
(300, 768)
(13, 768)
(73, 768)
(91, 768)
(33, 768)
(105, 768)
(247, 768)
(217, 768)
(97, 768)
(26, 768)
(101, 768)
(65, 768)
(44, 768)
(35, 768)
(57, 768)
(119, 768)
(274, 768)
(31, 768)
(139, 768)
(300, 768)
(61, 768)
(37, 768)
(64, 768)
(187, 768)
(300, 768)
(300, 768)
(300, 768)
(18, 768)
(116, 768)
(31, 768)
(11, 768)
(43, 768)
(45, 768)


Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (516 > 512). Running this sequence through BERT will result in indexing errors


(246, 768)
(217, 768)
(82, 768)
(190, 768)
(89, 768)
(39, 768)
(59, 768)
(22, 768)
(18, 768)
(131, 768)
(37, 768)
(117, 768)
(33, 768)
(53, 768)
(121, 768)
(281, 768)
(44, 768)
(112, 768)
(77, 768)
(77, 768)
(63, 768)
(61, 768)
(94, 768)
(45, 768)
(145, 768)
(47, 768)
(188, 768)
(44, 768)
(141, 768)
(44, 768)
(110, 768)
(122, 768)
(132, 768)
(300, 768)
(251, 768)
(95, 768)
(120, 768)
(43, 768)
(127, 768)
(179, 768)
(18, 768)
(26, 768)
(300, 768)
(118, 768)
(31, 768)
(27, 768)
(55, 768)
(300, 768)
(167, 768)
(110, 768)
(112, 768)
(219, 768)
(21, 768)
(41, 768)
(245, 768)
(51, 768)
(173, 768)
(26, 768)
(140, 768)
(114, 768)
(48, 768)
(44, 768)
(15, 768)
(87, 768)
(106, 768)
(28, 768)
(27, 768)
(145, 768)
(255, 768)
(200, 768)
(45, 768)
(136, 768)
(210, 768)
(43, 768)
(106, 768)
(178, 768)
(125, 768)
(300, 768)
(42, 768)
(46, 768)
(135, 768)
(227, 768)
(92, 768)
(31, 768)
(28, 768)
(63, 768)
(93, 768)
(127, 768)
(300, 768)
(108, 768)
(300, 768)
(84, 768)
(99, 768)
(26, 768)
(71, 768)
(14,

Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (677 > 512). Running this sequence through BERT will result in indexing errors


(66, 768)
(52, 768)
(50, 768)
(101, 768)
(158, 768)
(70, 768)
(78, 768)
(71, 768)
(300, 768)
(37, 768)
(300, 768)
(68, 768)
(91, 768)
(34, 768)
(143, 768)
(88, 768)
(48, 768)
(300, 768)
(142, 768)
(300, 768)
(72, 768)
(300, 768)
(45, 768)
(3, 768)
(102, 768)
(40, 768)
(70, 768)
(34, 768)
(80, 768)
(68, 768)
(36, 768)
(92, 768)
(26, 768)
(48, 768)
(300, 768)
(45, 768)
(261, 768)
(68, 768)
(139, 768)
(108, 768)
(73, 768)
(115, 768)
(21, 768)
(82, 768)
(269, 768)
(175, 768)
(18, 768)
(300, 768)
(49, 768)
(83, 768)
(127, 768)
(37, 768)
(28, 768)
(300, 768)
(23, 768)
(300, 768)
(69, 768)
(49, 768)
(86, 768)
(91, 768)
(300, 768)
(34, 768)
(224, 768)
(51, 768)
(170, 768)
(28, 768)
(48, 768)
(24, 768)
(57, 768)
(13, 768)
(300, 768)
(249, 768)
(256, 768)
(108, 768)
(300, 768)
(171, 768)


In [19]:
#문서 길이를 300으로 설정했음에도 sub_token 길이가 512를 초과하는 문서들은 제외 됨
import pickle

print('제외된 문서의 수: ',len(except_lst))

with open('except_lst.pkl', 'wb') as file:
    pickle.dump(except_lst, file)

print("except_lst를 피클 파일로 저장했습니다.")

제외된 문서의 수:  43
except_lst를 피클 파일로 저장했습니다.


In [20]:
#제외된 문서들의 길이를 150으로 재설정 후 다시 임베딩
new_except_lst=[]
to_txt_long_train=[]
for idx in except_lst:
    to_txt_long_train.append(to_txt_filter[idx])
    
to_txt_long_train=limit_words(to_txt_long_train,150)

for i in except_lst:
    try:
        cnt=0
        text = to_txt_long_train[cnt]
        cnt+=1
        embed_matrix=bert_word_embedding(text)   
        np.save('./bert_embedding/train(bert)/doc%d'%i,embed_matrix)
        print(embed_matrix.shape)
    except:
        new_except_lst.append(i)   #단어를 150개로 제한했는데도 sub_token 길이가 512가 넘는 문서는 new_except_lst에 추가

(150, 768)
(150, 768)
(150, 768)
(150, 768)
(150, 768)
(150, 768)
(150, 768)
(150, 768)
(150, 768)
(150, 768)
(150, 768)
(150, 768)
(150, 768)
(150, 768)
(150, 768)
(150, 768)
(150, 768)
(150, 768)
(150, 768)
(150, 768)
(150, 768)
(150, 768)
(150, 768)
(150, 768)
(150, 768)
(150, 768)
(150, 768)
(150, 768)
(150, 768)
(150, 768)
(150, 768)
(150, 768)
(150, 768)
(150, 768)
(150, 768)
(150, 768)
(150, 768)
(150, 768)
(150, 768)
(150, 768)
(150, 768)
(150, 768)
(150, 768)


In [21]:
new_except_lst

[]

In [22]:
#문서 길이를 300으로 설정했음에도 sub_token 길이가 512를 초과하는 문서들은 제외 됨
except_test_lst=[]

for i in range(len(x_test)):
    try:
        text = to_txt_filter[i+15062]
        embed_matrix=bert_word_embedding(text)       
        np.save('./bert_embedding/test(bert)/test%d'%i,embed_matrix)
        print(embed_matrix.shape)
    except:
        except_test_lst.append(i)   #sub_token 길이가 512가 넘는 문서는 except_lst에 추가

(21, 768)
(65, 768)
(90, 768)
(107, 768)
(107, 768)
(98, 768)
(99, 768)
(248, 768)
(83, 768)
(20, 768)
(33, 768)
(22, 768)
(45, 768)
(39, 768)
(107, 768)
(92, 768)
(67, 768)
(104, 768)
(99, 768)
(75, 768)
(79, 768)
(68, 768)
(77, 768)
(99, 768)
(56, 768)
(120, 768)
(41, 768)
(41, 768)
(67, 768)
(300, 768)
(119, 768)
(48, 768)
(43, 768)
(115, 768)
(82, 768)
(148, 768)
(28, 768)
(48, 768)
(247, 768)
(47, 768)
(276, 768)
(54, 768)
(50, 768)
(10, 768)
(43, 768)
(162, 768)
(32, 768)
(163, 768)
(97, 768)
(89, 768)
(300, 768)
(144, 768)
(33, 768)
(19, 768)
(22, 768)
(122, 768)
(106, 768)
(62, 768)
(152, 768)
(31, 768)
(86, 768)
(50, 768)
(142, 768)
(300, 768)
(83, 768)
(40, 768)
(101, 768)
(85, 768)
(150, 768)
(71, 768)
(43, 768)
(84, 768)
(18, 768)
(6, 768)
(91, 768)
(126, 768)
(29, 768)
(20, 768)
(51, 768)
(238, 768)
(19, 768)
(98, 768)
(63, 768)
(23, 768)
(31, 768)
(19, 768)
(27, 768)
(69, 768)
(104, 768)
(27, 768)
(63, 768)
(104, 768)
(300, 768)
(78, 768)
(26, 768)
(300, 768)
(73, 768)
(3

Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (694 > 512). Running this sequence through BERT will result in indexing errors


(121, 768)
(110, 768)
(79, 768)
(41, 768)
(36, 768)
(47, 768)
(99, 768)
(50, 768)
(31, 768)
(38, 768)
(180, 768)
(119, 768)
(107, 768)
(212, 768)
(300, 768)
(86, 768)
(51, 768)
(236, 768)
(247, 768)
(87, 768)
(93, 768)
(120, 768)
(22, 768)
(38, 768)
(101, 768)
(33, 768)
(68, 768)
(80, 768)
(169, 768)
(54, 768)
(71, 768)
(30, 768)
(50, 768)
(139, 768)
(41, 768)
(36, 768)
(138, 768)
(73, 768)
(149, 768)
(69, 768)
(21, 768)
(148, 768)
(52, 768)
(266, 768)
(37, 768)
(300, 768)


Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (677 > 512). Running this sequence through BERT will result in indexing errors


(93, 768)
(19, 768)
(61, 768)
(68, 768)
(54, 768)
(114, 768)
(16, 768)
(53, 768)
(186, 768)
(25, 768)
(155, 768)
(72, 768)
(67, 768)
(74, 768)
(79, 768)
(300, 768)
(130, 768)
(300, 768)
(63, 768)
(195, 768)


...

(50, 768)
(10, 768)
(121, 768)
(73, 768)
(57, 768)
(43, 768)
(27, 768)
(10, 768)


Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (656 > 512). Running this sequence through BERT will result in indexing errors


(300, 768)
(169, 768)
(22, 768)
(111, 768)
(121, 768)
(144, 768)
(95, 768)
(283, 768)
(21, 768)
(122, 768)
(27, 768)
(49, 768)
(299, 768)
(18, 768)
(146, 768)
(37, 768)
(159, 768)
(22, 768)
(41, 768)
(110, 768)
(44, 768)
(20, 768)
(34, 768)
(91, 768)
(196, 768)
(194, 768)
(78, 768)
(73, 768)
(38, 768)
(41, 768)
(104, 768)
(29, 768)
(46, 768)
(59, 768)
(64, 768)
(249, 768)
(233, 768)
(115, 768)
(128, 768)
(300, 768)
(46, 768)
(43, 768)
(54, 768)
(40, 768)


Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (1607 > 512). Running this sequence through BERT will result in indexing errors


(61, 768)
(150, 768)
(98, 768)
(154, 768)
(110, 768)
(34, 768)
(116, 768)
(85, 768)
(134, 768)
(105, 768)
(157, 768)
(109, 768)
(223, 768)
(69, 768)
(126, 768)
(75, 768)
(92, 768)
(163, 768)
(34, 768)
(5, 768)
(103, 768)
(19, 768)
(89, 768)
(24, 768)
(163, 768)
(183, 768)
(1, 768)
(59, 768)
(67, 768)
(71, 768)
(43, 768)
(96, 768)
(194, 768)
(79, 768)
(32, 768)
(48, 768)
(71, 768)
(59, 768)
(152, 768)
(5, 768)
(50, 768)
(205, 768)
(300, 768)
(30, 768)
(98, 768)
(74, 768)
(12, 768)
(114, 768)
(300, 768)
(85, 768)
(211, 768)
(145, 768)
(108, 768)
(94, 768)
(300, 768)
(181, 768)
(300, 768)
(43, 768)
(300, 768)
(58, 768)
(39, 768)
(99, 768)
(4, 768)
(39, 768)
(244, 768)
(291, 768)
(47, 768)
(56, 768)
(31, 768)
(28, 768)
(17, 768)
(285, 768)
(146, 768)
(43, 768)
(32, 768)
(57, 768)
(100, 768)
(55, 768)
(67, 768)
(37, 768)
(300, 768)
(18, 768)
(300, 768)
(13, 768)
(46, 768)
(120, 768)
(107, 768)
(26, 768)
(53, 768)
(143, 768)
(77, 768)
(137, 768)
(110, 768)
(300, 768)
(119, 768)
(221, 768)
(1

In [23]:
import pickle

print('제외된 문서의 수: ',len(except_test_lst))

with open('except_test_lst.pkl', 'wb') as file:
    pickle.dump(except_test_lst, file)

print("except_test_lst를 피클 파일로 저장했습니다.")

제외된 문서의 수:  13
except_test_lst를 피클 파일로 저장했습니다.


In [24]:
#제외된 문서들의 길이를 150으로 재설정 후 다시 임베딩
new_except_test_lst=[]
to_txt_long_test=[]
for idx in except_test_lst:
    to_txt_long_test.append(to_txt_filter[idx+15062])
    
to_txt_long_test=limit_words(to_txt_long_test,150)

for i in except_test_lst:
    try:
        cnt=0
        text = to_txt_long_test[cnt]
        cnt+=1
        embed_matrix=bert_word_embedding(text)       
        np.save('./bert_embedding/test(bert)/test%d'%i,embed_matrix)
        print(embed_matrix.shape)
    except:
        new_except_test_lst.append(i)   #단어를 150개로 제한했는데도 sub_token 길이가 512가 넘는 문서는 except_lst에 추가

(150, 768)
(150, 768)
(150, 768)
(150, 768)
(150, 768)
(150, 768)
(150, 768)
(150, 768)
(150, 768)
(150, 768)
(150, 768)
(150, 768)
(150, 768)


In [25]:
new_except_test_lst

[]